In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
from copy import deepcopy
import random
import matplotlib.pyplot as plt
import seaborn as sns
import calendar
#from orion.contrib.envs import load_env
#load_env()
import boto3
#from orion.sources import S3Source
aws_bucket = 'kilimanjaro-prod-datalake'
s3 = boto3.client('s3')

In [2]:
trans = pd.read_csv('transaction_tidy.csv', index_col=0)
trans.head()

/home/emmaranderson7/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,salestransactionkey,salesordernumber,orderdate,ordertime,shippingdate,customerkey,shipcountry,postcode,itemnumber,pricetype,discountpercent,unitssold,grosssales,grossprofit,year,month,dayofweek,sales_type,gsales_perunit
0,2139607,S0016-8920840,2015-08-30,13:00:00,2015-08-30,220c0261086d7d2c254a9ba8d9339e6c15cb2792,United Kingdom,S61,90956,Clearance,0.0,1,58.16,15.9233,2015,8,Sunday,S0,58.16
1,2842275,S0016-9272310,2016-03-10,15:19:00,2016-03-10,375083e129410daa79593eb12317e1a3ead57f59,United Kingdom,ML2,87518,Clearance,0.0,1,63.94,10.9850,2016,3,Thursday,S0,63.94
2,2821100,S0016-9263541,2016-03-03,20:24:00,2016-03-03,8151f4728247ec548b4178b0d0685f70f45f3ec4,United Kingdom,S25,89813,Full Price,0.0,1,4.32,0.0000,2016,3,Thursday,S0,4.32
3,3128532,S0016-9408798,2016-06-08,23:10:00,2016-06-09,d42a941336443fc3070d9ef2a6c36982bffc1932,United Kingdom,EH6,100726,Full Price,0.0,1,33.61,14.0900,2016,6,Wednesday,S0,33.61
4,2874597,S0016-9288483,2016-03-22,13:10:00,2016-03-22,f25eaaea0e1c9e713308d7a9ae2624589742e7d0,United Kingdom,PR4,95335,Clearance,0.0,1,56.76,12.4700,2016,3,Tuesday,S0,56.76


In [3]:
# To check bulk sales
trans2 = trans.groupby(
   ['customerkey', 'itemnumber']
).agg(
    {
         'unitssold': "sum",
        'grosssales':"sum"
    }
).round(decimals=2)
trans2 = trans2.reset_index()
trans2.head()

,customerkey,itemnumber,unitssold,grosssales
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2,105.85
1,0000010177de5b4048370276b036ff3ba06b7ae6,117352,2,29.66
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,87139,1,12.31
3,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,89814,1,5.14
4,0000089aee56e015b7de440137bbd90baf2b6039,89813,2,18.76


In [4]:
trans2['customerkey'].nunique()

2280365

In [5]:
trans2 = trans2.set_index(['customerkey', 'itemnumber'])

In [6]:
returns = pd.read_csv('returns_full.csv', index_col=0)
returns.head()

,salestransactionkey,salesordernumber,orderdate,ordertime,shippingdate,customerkey,shipcountry,postcode,itemnumber,pricetype,discountpercent,unitssold,grosssales,grossprofit,sales_type
852,4181333,SR202860,2017-01-26T00:00:00Z,1899-12-31T23:59:00Z,2017-01-26T00:00:00Z,19eb62cff114c93ff6e897b1076a0a01ca6e4c05,United Kingdom,SN15,108545,Clearance,0.0,-1,-15.36,-6.83,SR
856,11523864,SR228701,2017-05-25T00:00:00Z,1899-12-31T23:59:00Z,2017-05-26T00:00:00Z,fd93f85db5fe014871a5fc4f6baf48ec1feace58,United Kingdom,CA22,108316,Clearance,0.1,-1,-9.72,1.84,SR
1092,4151338,SR201182,2017-01-18T00:00:00Z,1899-12-31T23:59:00Z,2017-01-19T00:00:00Z,de1bbf0fb122352c7b11a8dc30ee270ed1ffe38b,United Kingdom,RH12,82137,Full Price,0.0,1,2.59,0.00,SR
3290,4258373,SR206753,2017-02-14T00:00:00Z,1899-12-31T23:59:00Z,2017-02-14T00:00:00Z,d08eac9b3191b2d0a49a6888ec85fbcd4d2810e3,United Kingdom,DN6,105484,Full Price,0.0,-1,-70.13,-44.54,SR
4239,4129194,SR199741,2017-01-13T00:00:00Z,1899-12-31T23:59:00Z,2017-01-13T00:00:00Z,7842a6a6bd6078ff1119efaff1c37a03d49634f2,United Kingdom,BT92,82137,Full Price,0.0,1,3.75,0.00,SR


In [7]:
# To check bulk sales
returns2 = returns.groupby(
   ['customerkey', 'itemnumber']
).agg(
    {
         'unitssold': "sum",
        'grosssales':"sum"
        
    }
).round(decimals=2)
returns2 = returns2.reset_index()
returns2.head()

,customerkey,itemnumber,unitssold,grosssales
0,0000010177de5b4048370276b036ff3ba06b7ae6,117352,-2,-2.65
1,000021af723428ee04a3a9232660d8808389c1d5,124819,-3,-268.47
2,000021af723428ee04a3a9232660d8808389c1d5,124826,-3,-226.40
3,00003c325d651344e14da4ae94b55de9b3cf92f1,82137,3,23.29
4,00003c325d651344e14da4ae94b55de9b3cf92f1,125505,-3,-420.45


In [8]:
returns2['unitssold'] = returns2['unitssold'].abs()
returns2['grosssales'] = returns2['grosssales'].abs()

In [9]:
returns2 = returns2.rename(columns={"unitssold": "unitsreturned", "grosssales":"grossret"})

In [10]:
returns2 = returns2[['customerkey','itemnumber','unitsreturned', 'grossret']]

In [11]:
returns2.head(1)

,customerkey,itemnumber,unitsreturned,grossret
0,0000010177de5b4048370276b036ff3ba06b7ae6,117352,2,2.65


In [12]:
full_df = trans2.merge(returns2, how='left', on=['customerkey','itemnumber'])

In [13]:
full_df['customerkey'].nunique()

2280365

In [14]:
full_df.head()

,customerkey,itemnumber,unitssold,grosssales,unitsreturned,grossret
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2,105.85,NaN,NaN
1,0000010177de5b4048370276b036ff3ba06b7ae6,117352,2,29.66,2.0,2.65
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,87139,1,12.31,NaN,NaN
3,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,89814,1,5.14,NaN,NaN
4,0000089aee56e015b7de440137bbd90baf2b6039,89813,2,18.76,NaN,NaN


In [15]:
full_df = full_df.replace(np.nan, 0)

In [16]:
full_df['unitsreturned']= full_df['unitsreturned'].abs()
full_df.head()

,customerkey,itemnumber,unitssold,grosssales,unitsreturned,grossret
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2,105.85,0.0,0.00
1,0000010177de5b4048370276b036ff3ba06b7ae6,117352,2,29.66,2.0,2.65
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,87139,1,12.31,0.0,0.00
3,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,89814,1,5.14,0.0,0.00
4,0000089aee56e015b7de440137bbd90baf2b6039,89813,2,18.76,0.0,0.00


In [17]:
full_df['unitssold'] = full_df['unitssold'] - full_df['unitsreturned']
full_df['grosssales'] = full_df['grosssales'] - full_df['grossret']

In [18]:
full_df.head()

,customerkey,itemnumber,unitssold,grosssales,unitsreturned,grossret
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2.0,105.85,0.0,0.00
1,0000010177de5b4048370276b036ff3ba06b7ae6,117352,0.0,27.01,2.0,2.65
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,87139,1.0,12.31,0.0,0.00
3,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,89814,1.0,5.14,0.0,0.00
4,0000089aee56e015b7de440137bbd90baf2b6039,89813,2.0,18.76,0.0,0.00


In [19]:
full_df= full_df[['customerkey','itemnumber','unitssold','grosssales']]

In [20]:
full_df = full_df[(full_df['unitssold']>0)]

In [21]:
units = full_df.sort_values("unitssold", ascending=False)

In [22]:
units.head(1)

,customerkey,itemnumber,unitssold,grosssales
595443,17c97a02c3f208c0f66365adafef6dd0219c3df8,89814,134.0,1215.6


In [23]:
units['customerkey'].nunique()

2229189

In [24]:
units.to_csv('units_sorted.csv')

In [25]:
cust_units = units.groupby(
   ['customerkey']
).agg(
    { 
        'itemnumber':lambda x: x.nunique(),
         'unitssold': "sum",
         'grosssales': "sum",
    }
).round(decimals=2)
cust_units = cust_units.reset_index()
cust_units

,customerkey,itemnumber,unitssold,grosssales
0,0000010177de5b4048370276b036ff3ba06b7ae6,1,2.0,105.85
1,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,2,2.0,17.45
2,0000089aee56e015b7de440137bbd90baf2b6039,4,5.0,115.59
3,000009008567d4740aec3d882e67fe4d2d74eb39,3,3.0,253.48
4,0000167ab10ff470d5b1c2116b9aa33ddccc12bd,3,3.0,112.01
...,...,...,...,...
2229184,ffffdf2d6e78c31b7669022a96d6e8ab0f076404,2,2.0,57.95
2229185,ffffe0a8c1a504232cebca86d2be5e3f59cb9f92,6,7.0,271.06
2229186,ffffe0b8b360367113c4f1e3d88a9ceb14c031aa,2,4.0,416.38
2229187,fffff36d72733a953ace2a324b8b231502186204,2,2.0,33.86


In [26]:
of = pd.read_csv('customer_orderfreq.csv', index_col=0)
of.head()

/home/emmaranderson7/.local/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,customerkey,orders,unitssold,grosssales,gsales_perunit
2238223,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,61.44
895876,649cca818138ed8152825855e0538ed7360bd55a,152,400,18320.26,51.97
762129,559aec727d6d73e55e476a34a374f1fb26dc06cf,152,401,16730.55,44.44
478420,35d17f77612459a698d0179b7525c6eedacc6fd6,144,469,15046.12,37.51
2137077,efef72470d6b8bc80f22afd9192a39f26357095f,135,263,16536.17,68.60


In [27]:
customers_obj = 'masters/uploads/customers/1560425511130_Peak_customers.csv'
customers = s3.get_object(Bucket=aws_bucket, Key=customers_obj)
customers = pd.read_csv(customers['Body'])

In [36]:
summary = of.merge(customers, how='left', on='customerkey')
summary.head()

,customerkey,orders,unitssold,grosssales,gsales_perunit,loyaltyaccount,gender
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,61.44,Yes,female
1,649cca818138ed8152825855e0538ed7360bd55a,152,400,18320.26,51.97,Yes,female
2,559aec727d6d73e55e476a34a374f1fb26dc06cf,152,401,16730.55,44.44,Yes,female
3,35d17f77612459a698d0179b7525c6eedacc6fd6,144,469,15046.12,37.51,Yes,female
4,efef72470d6b8bc80f22afd9192a39f26357095f,135,263,16536.17,68.60,NaN,NaN


In [37]:
summary = summary.rename(columns={"grosssales": "total_grosssales", "gsales_perunit":"ASP" })
summary.head(1)

,customerkey,orders,unitssold,total_grosssales,ASP,loyaltyaccount,gender
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,61.44,Yes,female


In [70]:
summary = summary.rename(columns={"unitssold": "total_unitssold"})
summary.head(1)

,customerkey,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender
0,fb484d6a54da0d56910e2df0064c18c722801a66,256,603,35645.08,61.44,Yes,female


In [71]:
summary.to_csv('cust_summary.csv')

In [45]:
summary = summary[['customerkey','orders','total_unitssold', 'total_grosssales','ASP', 'loyaltyaccount', 'gender']]

In [46]:
full = cust_units.merge(summary, how='left', on='customerkey')
full.head()

,customerkey,itemnumber,unitssold,grosssales,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender
0,0000010177de5b4048370276b036ff3ba06b7ae6,1,2.0,105.85,1,4,135.51,33.88,No,female
1,0000010177de5b4048370276b036ff3ba06b7ae6,1,2.0,105.85,1,4,135.51,33.88,No,female
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,2,2.0,17.45,1,2,17.45,8.72,No,female
3,0000089aee56e015b7de440137bbd90baf2b6039,4,5.0,115.59,2,5,115.59,22.28,Yes,female
4,000009008567d4740aec3d882e67fe4d2d74eb39,3,3.0,253.48,1,3,253.48,84.49,No,female


In [47]:
full = full.rename(columns={"itemnumber": "total_items"})

In [49]:
full_df = full_df[['customerkey','itemnumber']]

In [50]:
alldf = full_df.merge(full, how='left', on='customerkey')

In [51]:
alldf.head()

,customerkey,itemnumber,total_items,unitssold,grosssales,orders,total_unitssold,total_grosssales,ASP,loyaltyaccount,gender
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,1,2.0,105.85,1,4,135.51,33.88,No,female
1,0000010177de5b4048370276b036ff3ba06b7ae6,116075,1,2.0,105.85,1,4,135.51,33.88,No,female
2,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,87139,2,2.0,17.45,1,2,17.45,8.72,No,female
3,0000028fc7e0b307e0ff9dcdf3e182b1ff70ae60,89814,2,2.0,17.45,1,2,17.45,8.72,No,female
4,0000089aee56e015b7de440137bbd90baf2b6039,89813,4,5.0,115.59,2,5,115.59,22.28,Yes,female


In [35]:
alldf = alldf.rename(columns={"grosssales": "total_grosssales", "gsales_perunit":"ASP" })
alldf.head(1)

,customerkey,itemnumber,total_items,unitssold,total_grosssales,orders,ASP,loyaltyaccount,gender
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,1,2.0,105.85,1,33.88,No,female


In [52]:
alldf = alldf[['customerkey','itemnumber','unitssold','total_grosssales','total_items','total_unitssold','orders','ASP','loyaltyaccount','gender']]

In [53]:
prod_obj = 'masters/uploads/product/1560425499995_Peak_product.csv'
product = s3.get_object(Bucket=aws_bucket, Key=prod_obj)
product = pd.read_csv(product['Body'])
product.head()

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (0,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,itemnumber,brandcode,category,colourvalue,divisioncode,fabric,itemcategorycode,itemcolour,itemfamilycode,itemquarter,itemseason,itemstylecode,productgroup
0,041333,PO,Mens,BLACK,FOOTWEAR,Unknown,FM,NaN,FA,Quarter 3,AW,41333,Vulcanised
1,083567,FEK,Mens,BLUE,APPAREL,Unknown,AM,Blue,WB,Quarter 2,SS,83566,T-shirts
2,033641,FM,Mens,WHITE,APPAREL,Unknown,AM,NaN,FA,Quarter 3,AW,33641,T-shirts
3,066348,L7,Womens,STD,APPAREL,Unknown,AW,NaN,DX,Quarter 3,AW,66348,Skirts
4,028949,C9,Mens,BLACK,APPAREL,Unknown,AM,NaN,FA,Quarter 2,SS,28949,T-shirts


In [54]:
product = product[['itemnumber','brandcode','category','divisioncode','productgroup']]
product['itemnumber'] = pd.to_numeric(product['itemnumber'], errors='coerce')
product.head(1)

,itemnumber,brandcode,category,divisioncode,productgroup
0,41333.0,PO,Mens,FOOTWEAR,Vulcanised


In [59]:
full_test = alldf.merge(product, how='left', on='itemnumber')
full_test.head(2)

,customerkey,itemnumber,unitssold,total_grosssales,total_items,total_unitssold,orders,ASP,loyaltyaccount,gender,brandcode,category,divisioncode,productgroup
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2.0,135.51,1,4,1,33.88,No,female,NN,Junior,FOOTWEAR,Nylon Running
1,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2.0,135.51,1,4,1,33.88,No,female,NN,Junior,FOOTWEAR,Nylon Running


In [60]:
full_test = full_test[['customerkey','itemnumber','unitssold','brandcode','category','divisioncode','productgroup','orders','total_grosssales','total_items','total_unitssold','ASP','loyaltyaccount','gender']]
full_test.head(2)

,customerkey,itemnumber,unitssold,brandcode,category,divisioncode,productgroup,orders,total_grosssales,total_items,total_unitssold,ASP,loyaltyaccount,gender
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2.0,NN,Junior,FOOTWEAR,Nylon Running,1,135.51,1,4,33.88,No,female
1,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2.0,NN,Junior,FOOTWEAR,Nylon Running,1,135.51,1,4,33.88,No,female


In [65]:
full_test['unitssold'] = full_test['unitssold'].astype('int64')
full_test.head(1)

,customerkey,itemnumber,unitssold,brandcode,category,divisioncode,productgroup,orders,total_grosssales,total_items,total_unitssold,ASP,loyaltyaccount,gender
0,0000010177de5b4048370276b036ff3ba06b7ae6,116075,2,NN,Junior,FOOTWEAR,Nylon Running,1,135.51,1,4,33.88,No,female


In [66]:
full_test.isnull().sum()

customerkey               0
itemnumber                0
unitssold                 0
brandcode            908393
category                  0
divisioncode              0
productgroup              0
orders                    0
total_grosssales          0
total_items               0
total_unitssold           0
ASP                       0
loyaltyaccount      1800273
gender              1998418
dtype: int64

In [67]:
full_test['brandcode'] = full_test['brandcode'].fillna('none')

In [68]:
loyal_check = full_test[(full_test['loyaltyaccount'].isnull())]
loyal_check.head(1)

,customerkey,itemnumber,unitssold,brandcode,category,divisioncode,productgroup,orders,total_grosssales,total_items,total_unitssold,ASP,loyaltyaccount,gender
18,000034c0f49407a1d7cd7c39ce639664df1d6e8f,122839,2,NN,Womens,FOOTWEAR,Nylon Running,2,215.47,2,2,107.74,NaN,NaN


In [69]:
loyal_remove = loyal_check['customerkey'].values
loyal_remove

array(['000034c0f49407a1d7cd7c39ce639664df1d6e8f',
       '000034c0f49407a1d7cd7c39ce639664df1d6e8f',
       '0000694f7e844466db9347b56f3d12cc5de71241', ...,
       'ffffe0a8c1a504232cebca86d2be5e3f59cb9f92',
       'fffff36d72733a953ace2a324b8b231502186204',
       'fffff36d72733a953ace2a324b8b231502186204'], dtype=object)

In [72]:
full_test = full_test[~full_test.customerkey.isin(loyal_remove)]

In [76]:
full_test = deepcopy(full_test)

In [78]:
len(full_test)

5991660

In [77]:
full_test['gender'] = full_test['gender'].fillna('unknown')

In [79]:
full_test.to_csv('Product_customer.csv')